# Figure 1

## Create the AR(2) models

In [ ]:
import statsmodels.api as sm
import numpy as np
import scipy as sp
import scipy.io as io
import matplotlib.pyplot as plt
from matplotlib.backends.backend_pdf import PdfPages

# Set AR(2) parameters.
ar1   = np.array([1,  0.0, 0.99])     # High frequency
ar2   = np.array([1, -1.4, 0.99])     # Low frequency (1:2)
arPhi = np.array([1, -1.17, 0.99])    # Low frequency (1:Phi)
ar0   = np.array([1, -0.8])           # Background 1/f

# Build AR(2) processes.
arma_process_1   = sm.tsa.ArmaProcess(ar1,   [1])
arma_process_2   = sm.tsa.ArmaProcess(ar2,   [1])
arma_process_phi = sm.tsa.ArmaProcess(arPhi, [1])
arma_process_0   = sm.tsa.ArmaProcess(ar0,   [1])

# Simulate example AR(2) processes.
y1   = arma_process_1.generate_sample(250)
y2   = arma_process_2.generate_sample(250)
yphi = arma_process_phi.generate_sample(250)
y0   = arma_process_0.generate_sample(250)

# Plot example traces.
f = plt.figure(figsize=(7, 5), dpi=80)
plt.subplot(411); plt.plot(y1);               plt.title('High Frequency Signal');   plt.axis('off')
plt.subplot(412); plt.plot(y2,'g');           plt.title('Low Frequency Signal (1:2)');   plt.axis('off')
plt.subplot(413); plt.plot(yphi,'xkcd:gold'); plt.title(r'Low Frequency Signal (1:$\phi$)'); plt.axis('off')
plt.subplot(414); plt.plot(y0,'k');           plt.title('Background 1/f');   plt.axis('off');
f.savefig("Figure-1A.pdf", bbox_inches='tight')

## Compute spectra for example traces

In [ ]:
# Use the multitaper method.
import nitime.algorithms as tsa
NW = 3                                   # Define the normalized time-bandwidth product.
dt = 1;                                  # Arbitrary sample interval
faxis, S1, _   = tsa.multi_taper_psd(y1 - y1.mean(), Fs=1/dt, NW=NW);
faxis, S2, _   = tsa.multi_taper_psd(y2 - y2.mean(), Fs=1/dt, NW=NW);
faxis, SPhi, _ = tsa.multi_taper_psd(yphi - yphi.mean(), Fs=1/dt, NW=NW);
faxis, S0, _   = tsa.multi_taper_psd(y0 - y0.mean(), Fs=1/dt, NW=NW);

# Plot the spectra.
f = plt.figure(figsize=(7, 5), dpi=80)
plt.semilogy(faxis[:100], S1[:100],               label='High Frequency Signal')
plt.semilogy(faxis[:100], S2[:100],  'g',         label='Low Frequency Signal (1:2)')
plt.semilogy(faxis[:100], SPhi[:100],'xkcd:gold', label=r'Low Frequency Signal (1:$\phi$)')
plt.semilogy(faxis[:100], S0[:100],  'k',         label='Background 1/f')
plt.xlabel('Frequency [Hz]')
plt.ylabel('Power [dB]')
plt.grid()
plt.legend(loc='lower left')
plt.show()
f.savefig("Figure-1B.pdf", bbox_inches='tight')

## Define the phase-locking value (PLV)

From [Palva JM, Palva S, Kaila K. *Phase synchrony among neuronal oscillations in the human cortex.* J Neurosci. 2005](https://pubmed.ncbi.nlm.nih.gov/15829648/)

In [ ]:
def PLV(m, Y1, Y2, win_size):
    # m  = frequency ratio
    # Y1 = high frequency signal
    # Y2 = low frequency signal
    # win_size = duration of signal used to evaluate phase locking.
    phase_HF = np.transpose(np.angle(sp.signal.hilbert(np.transpose(Y1))))
    phase_LF = np.transpose(np.angle(sp.signal.hilbert(np.transpose(Y2))))
    # Compute PLV
    PLV = np.zeros(np.shape(Y1)[0]-win_size)
    for k in np.arange(np.shape(Y1)[0]-win_size):
        pL = phase_LF[k:k+win_size]
        pH = phase_HF[k:k+win_size]
        PLV[k] = 1/win_size * np.abs(np.sum(np.exp(1j*(m*pL-pH))))
    # Compute shuffled PLV
    PLV_surr = np.empty(200)
    for k in np.arange(200):
        i0 = np.random.randint(np.shape(Y1)[0]-win_size)
        pL = phase_LF[i0:i0+win_size]
        i0 = np.random.randint(np.shape(Y1)[0]-win_size)
        pH = phase_HF[i0:i0+win_size]
        PLV_surr[k] = 1/win_size * np.abs(np.sum(np.exp(1j*(m*pL-pH))))

    # Compute the proportion of PLV with p <= 0.01.
    proportion_detected = np.sum(PLV > 2.42*np.mean(PLV_surr))/np.shape(Y1)[0]
    
    return proportion_detected

## Perform simulation and compute proportion of significant PLV

This is slow, and counts from 0 to `J`-1 (=999).

In [ ]:
J        = 1000;  # Generate J signals.
win_size = 50;    # Window size for simulated AR(2).
m        = 1.618;     # Define the PLV frequency ratio.

proportion_detected_12   = np.empty(J)
proportion_detected_1Phi = np.empty(J)

for j in np.arange(J):                             # For each signal, 
    print(j,end=' ')                               # ...create 100 instances of duration win_size,
    Y1  = np.empty([100,win_size])                 # ...for High freq signal,
    Y2  = np.empty([100,win_size])                 # ...for Low freq (1:2) signal,
    YPhi= np.empty([100,win_size])                 # ...for Low freq (1:Phi) signal.
    for k in np.arange(100):                       # For each instance k,
        if np.random.rand()>0.9:                   # ...10% of the time,
            y1   = arma_process_1.generate_sample(win_size)  # ...y1 is   High freq signal,
            y2   = arma_process_2.generate_sample(win_size)  # ...y2 is   Low freq (1:2) signal,
            yphi = arma_process_phi.generate_sample(win_size)# ...yphi is Low freq (1:Phi) signal,
        else:                                      # ... 90% of the time,
            y1   = arma_process_0.generate_sample(win_size)  # ...y1,
            y2   = arma_process_0.generate_sample(win_size)  # ...y2,
            yphi = arma_process_0.generate_sample(win_size)  # ...and yPhi are all background 1/f.
        Y1[k,:]   = y1
        Y2[k,:]   = y2
        YPhi[k,:] = yphi
                                                   # Determine proportion of significant PLV detections.    
    proportion_detected_12[j]   = PLV(m, Y1.reshape(100*win_size,1), Y2.reshape(100*win_size,1), win_size)
    proportion_detected_1Phi[j] = PLV(m, Y1.reshape(100*win_size,1), YPhi.reshape(100*win_size,1), win_size)

## Display the resulting proportion of significant PLV detections as histograms

In [ ]:
f    = plt.figure(figsize=(7, 5), dpi=80)
bins = np.linspace(0,0.15,20)
plt.hist(proportion_detected_12,  bins, density=True, rwidth=0.75, align='left', color='g',      label='1:2')
plt.hist(proportion_detected_1Phi,bins, density=True, rwidth=0.75, align='mid',color='xkcd:gold', label=r'1:$\phi$')
plt.legend()
plt.xlabel('Proportion significant PLV')
plt.ylabel('Density');
plt.title('PLV with m = '+str(m))
f.savefig('Figure-1C-m-'+str(m)+'.pdf', bbox_inches='tight')

print(sp.stats.ttest_ind(proportion_detected_1Phi,proportion_detected_12,equal_var=False))